In [1]:
import math
import time
import csv
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']="1"
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import numpy as np
import torch 
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F

torch.manual_seed(105)
np.random.seed(42)
np.set_printoptions(suppress=True, precision=3, threshold=40, linewidth=100, edgeitems=20)  

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
#torch.backends.cudnn.enabled = False 
#Note:This cmd will lead to illegal memory runtime error (or sth. else concerning memory) if added...
#Note:But it doesn't hurt on the other server, and the other models... damns

import itertools
from tqdm import trange, tqdm
import matplotlib.pyplot as plt
import importlib
%matplotlib inline

'''
If curious about the detailed realization of certain module(s) below, try:
    print(*inspect.getsource(Module).split('\n'), sep='\n')
to get its source code.
'''
from Model.DeepCNN import ResNet
from Train.train_v1 import train
from Datas.data_load import dataset_append
from Test.Evaluate import evaluate
from utils.Save_and_load import save, load
from configs import opt, res_trans, RDevelop

In [2]:
def Model_def(opt, Channels, device_train, load_model):
    '''
    Define (and load saved) model 
    '''
    model = ResNet(Channels, bias=True)
    model.initialize()
    if load_model:
        model = load(model, opt.load_model_path)
    print(model.parameters)
    
    if device_train.type == 'cuda':
        torch.cuda.empty_cache()
        model = model.to(device_train)
        print("allocated and reserved GPU memory: {}, {} (GiB)".format(
                torch.cuda.memory_allocated()/(1024**3), torch.cuda.memory_reserved()/(1024**3)))
    return model

In [3]:
Channels = [2, 4, 6]
device_train = torch.device('cuda:0') 
device_test = torch.device('cuda:0')
ckpt_name = "ResNet.pth"
opt.load_model_path = opt.modelpath+ckpt_name
model = Model_def(opt, Channels, device_train, False)

Net Initialized
<bound method Module.parameters of ResNet(
  (conv1): Sequential(
    (0): Conv3d(1, 2, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (reslayers): Sequential(
    (0): Sequential(
      (0): BasicBlock(
        (conv): Sequential(
          (0): Conv3d(2, 2, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), padding_mode=replicate)
          (1): LeakyReLU(negative_slope=0.01, inplace=True)
          (2): Conv3d(2, 2, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), padding_mode=replicate)
        )
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv): Sequential(
          (0): Conv3d(2, 2, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), padding_mode=replicate)
          (1): LeakyReLU(negative_slope=0.01, inplace=True)
          (2): Conv3d(2, 2, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), padding_mode=replicate)
        )

In [4]:
dataset_train = dataset_append(opt, opt.dataidx_train, res_trans)
dataset_test = dataset_append(opt, opt.dataidx_test, res_trans) 

  0%|          | 0/8 [00:00<?, ?it/s]

../dataset_train_v1/0000Acid_ini Read in 0.70 s
../dataset_train_v1/0000Inhibitor Read in 0.73 s
--> Mask 0 Read Complete, with dataset size: (1, 1, 80, 1000, 1000), 9 masks left ...

../dataset_train_v1/0001Acid_ini Read in 0.72 s
../dataset_train_v1/0001Inhibitor Read in 0.69 s
--> Mask 1 Read Complete, with dataset size: (2, 1, 80, 1000, 1000), 8 masks left ...

../dataset_train_v1/0002Acid_ini Read in 0.68 s
../dataset_train_v1/0002Inhibitor Read in 0.70 s
--> Mask 2 Read Complete, with dataset size: (3, 1, 80, 1000, 1000), 7 masks left ...

../dataset_train_v1/0003Acid_ini Read in 0.71 s
../dataset_train_v1/0003Inhibitor Read in 0.68 s
--> Mask 3 Read Complete, with dataset size: (4, 1, 80, 1000, 1000), 6 masks left ...

../dataset_train_v1/0004Acid_ini Read in 0.71 s
../dataset_train_v1/0004Inhibitor Read in 0.72 s
--> Mask 4 Read Complete, with dataset size: (5, 1, 80, 1000, 1000), 5 masks left ...

../dataset_train_v1/0005Acid_ini Read in 0.72 s
../dataset_train_v1/0005Inhibito

 12%|█▎        | 1/8 [00:34<04:01, 34.49s/it]

--> Mask 9 Read Complete, with dataset size: (10, 1, 80, 1000, 1000), 0 masks left ...

../dataset_train_v1/0010Acid_ini Read in 0.73 s
../dataset_train_v1/0010Inhibitor Read in 0.73 s
--> Mask 10 Read Complete, with dataset size: (1, 1, 80, 1000, 1000), 9 masks left ...

../dataset_train_v1/0011Acid_ini Read in 0.70 s
../dataset_train_v1/0011Inhibitor Read in 0.72 s
--> Mask 11 Read Complete, with dataset size: (2, 1, 80, 1000, 1000), 8 masks left ...

../dataset_train_v1/0012Acid_ini Read in 0.75 s
../dataset_train_v1/0012Inhibitor Read in 0.73 s
--> Mask 12 Read Complete, with dataset size: (3, 1, 80, 1000, 1000), 7 masks left ...

../dataset_train_v1/0013Acid_ini Read in 0.67 s
../dataset_train_v1/0013Inhibitor Read in 0.69 s
--> Mask 13 Read Complete, with dataset size: (4, 1, 80, 1000, 1000), 6 masks left ...

../dataset_train_v1/0014Acid_ini Read in 0.75 s
../dataset_train_v1/0014Inhibitor Read in 0.67 s
--> Mask 14 Read Complete, with dataset size: (5, 1, 80, 1000, 1000), 5 mas

 25%|██▌       | 2/8 [01:09<03:28, 34.69s/it]

...

../dataset_train_v1/0080Acid_ini Read in 0.68 s
../dataset_train_v1/0080Inhibitor Read in 0.69 s
--> Mask 80 Read Complete, with dataset size: (1, 1, 80, 1000, 1000), 9 masks left ...

../dataset_train_v1/0081Acid_ini Read in 0.69 s
../dataset_train_v1/0081Inhibitor Read in 0.66 s
--> Mask 81 Read Complete, with dataset size: (2, 1, 80, 1000, 1000), 8 masks left ...

../dataset_train_v1/0082Acid_ini Read in 0.68 s
../dataset_train_v1/0082Inhibitor Read in 0.70 s
--> Mask 82 Read Complete, with dataset size: (3, 1, 80, 1000, 1000), 7 masks left ...

../dataset_train_v1/0083Acid_ini Read in 0.68 s
../dataset_train_v1/0083Inhibitor Read in 0.68 s
--> Mask 83 Read Complete, with dataset size: (4, 1, 80, 1000, 1000), 6 masks left ...

../dataset_train_v1/0084Acid_ini Read in 0.65 s
../dataset_train_v1/0084Inhibitor Read in 0.64 s
--> Mask 84 Read Complete, with dataset size: (5, 1, 80, 1000, 1000), 5 masks left ...

../dataset_train_v1/0085Acid_ini Read in 0.65 s
../dataset_train_v1/0085Inh

 50%|█████     | 1/2 [00:33<00:33, 33.66s/it]

--> Mask 89 Read Complete, with dataset size: (10, 1, 80, 1000, 1000), 0 masks left ...

../dataset_train_v1/0090Acid_ini Read in 0.67 s
../dataset_train_v1/0090Inhibitor Read in 0.68 s
--> Mask 90 Read Complete, with dataset size: (1, 1, 80, 1000, 1000), 9 masks left ...

../dataset_train_v1/0091Acid_ini Read in 0.64 s
../dataset_train_v1/0091Inhibitor Read in 0.63 s
--> Mask 91 Read Complete, with dataset size: (2, 1, 80, 1000, 1000), 8 masks left ...

../dataset_train_v1/0092Acid_ini Read in 0.64 s
../dataset_train_v1/0092Inhibitor Read in 0.66 s
--> Mask 92 Read Complete, with dataset size: (3, 1, 80, 1000, 1000), 7 masks left ...

../dataset_train_v1/0093Acid_ini Read in 0.67 s
../dataset_train_v1/0093Inhibitor Read in 0.68 s
--> Mask 93 Read Complete, with dataset size: (4, 1, 80, 1000, 1000), 6 masks left ...

../dataset_train_v1/0094Acid_ini Read in 0.12 s
../dataset_train_v1/0094Inhibitor Read in 0.14 s
--> Mask 94 Read Complete, with dataset size: (5, 1, 80, 1000, 1000), 5 ma

100%|██████████| 2/2 [01:08<00:00, 34.19s/it]

--> Mask 99 Read Complete, with dataset size: (10, 1, 80, 1000, 1000), 0 masks left ...



In [5]:
def opt_strategy(model, opt):
    LR = 1e-3
    optimizer = torch.optim.Adam(model.parameters(), lr=LR,)# weight_decay=1e-4)
    scheduler = None
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.7)
    return optimizer, scheduler 

In [6]:
TotLoss = nn.MSELoss()
opt.epochs = 800
optimizer, scheduler = opt_strategy(model, opt)
train_log, test_log, lr_log = train(model, optimizer, scheduler, opt, dataset_train, device_train,
                                           dataset_test, res_trans, ckpt_name, TotLoss=TotLoss)
if not os.path.exists(opt.load_model_path):
    save(model, opt.modelpath, ckpt_name)

  0%|          | 1/800 [02:23<31:48:10, 143.29s/it]

Epoch 0 | LR: 1.00e-03 | Train Loss: 6.22e-02 | Test Loss: 1.54e+01, 4.72e-02, 97.12%


  3%|▎         | 21/800 [28:21<17:21:38, 80.23s/it]

Epoch 20 | LR: 1.00e-03 | Train Loss: 4.46e-03 | Test Loss: 1.82e+01, 7.45e-02, 98.85%


  5%|▌         | 41/800 [54:25<16:54:07, 80.17s/it]

...

 95%|█████████▌| 761/800 [16:32:33<52:10, 80.27s/it]  

Epoch 760 | LR: 8.24e-05 | Train Loss: 1.95e-03 | Test Loss: 1.01e+01, 3.81e-02, 64.10%


 98%|█████████▊| 781/800 [16:58:38<25:24, 80.26s/it]

Epoch 780 | LR: 8.24e-05 | Train Loss: 4.87e-04 | Test Loss: 1.05e+01, 5.60e-02, 64.18%


100%|██████████| 800/800 [17:23:18<00:00, 78.25s/it]


In [4]:
model = Model_def(opt, Channels, device_train, True)
Runtime, RMSE, NMSE, _, CD_err_x, CD_err_y, CD_RMSE = evaluate(opt, model, device_train, res_trans, RDevelop)
print("Mean runtime:", np.mean(Runtime))
print("RMSE(Ihb, rate):", np.mean(RMSE,axis=-1))
print("NRMSE(Ihb, rate):", np.mean(NMSE,axis=-1))
print("CD Error(x, y):", np.mean(CD_RMSE,axis=-1))

Net Initialized
!Model load from  ./Ckpts/ResNet.pth
<bound method Module.parameters of ResNet(
  (conv1): Sequential(
    (0): Conv3d(1, 2, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (reslayers): Sequential(
    (0): Sequential(
      (0): BasicBlock(
        (conv): Sequential(
          (0): Conv3d(2, 2, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), padding_mode=replicate)
          (1): LeakyReLU(negative_slope=0.01, inplace=True)
          (2): Conv3d(2, 2, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), padding_mode=replicate)
        )
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv): Sequential(
          (0): Conv3d(2, 2, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), padding_mode=replicate)
          (1): LeakyReLU(negative_slope=0.01, inplace=True)
          (2): Conv3d(2, 2, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 

  0%|          | 0/20 [00:00<?, ?it/s]

../dataset_train_v1/0080Acid_ini Read in 0.12 s
../dataset_train_v1/0080Inhibitor Read in 0.12 s
RMSE for label 0: [0.01  0.866]
NMSE for label 0: [11.629  2.176]
NMSE_height for label 0: [12.403  1.916]


<<<
time:	0.435943s
<<<
<<<
time:	0.39695s
<<<



!Start to clear memory
CD_x error at height 0:
 [ 2.3    0.2    2.347  2.9    2.     1.83   2.1   -0.2    0.5    0.6    0.372 -0.     0.2    2.1
  0.8    0.2    0.3   -0.     1.4  ]
CD_y error at height 0:
 [-2.7    1.4    1.1    1.8    0.153  0.9    0.9    1.2   -0.1   -0.4    0.059  4.9   -0.7   -0.049
  0.575  0.2    0.2   -0.     0.3  ]
CD_x error at height 8:
 [-0.    -0.     1.6   -0.1    1.8    0.3    1.2   -0.1   -0.1    0.3   -0.4   -0.     0.5   -0.
  5.7    0.133 38.102 -0.2   -1.5    1.7  ]
CD_y error at height 8:
 [ 0.2   -0.6    1.318  1.5    0.461  2.346  0.941  2.     0.133  0.1   -0.818 -0.063  0.1   -0.1
 -0.1   -0.1    0.041  0.1    0.5  ]
CD_x error at height 16:
 [-0.2   -0.4    0.322  0.2   -0.    -0.079  0.5   -0.12   0.1    0.6    0.3    0.084  0.8    0.1
  0.1    0.237  0.2    0.9    2.   ]
CD_y error at height 16:
 [ 0.2   -0.9    0.313 -0.    -0.085 -0.3    0.1   -0.1   -0.9    0.946  0.18  -0.339  0.148 -1.063
  2.6   -0.133 -0.1    0.529  1.727]
CD_x error

...

<<<
time:	0.365029s
<<<


CD_x error at height 0:
 [1.7 1.5 1.6 2.3 0.2]
CD_y error at height 0:
 [0.7   0.232 0.416 0.228 2.   ]
CD_x error at height 8:
 [ 0.3  0.4  0.5 -0.   0.1]
CD_y error at height 8:
 [ 0.4   -1.43  -0.2    0.038  0.043]
CD_x error at height 16:
 [ 1.7  3.3 -0.9  0.  -0.2]
CD_y error at height 16:
 [-1.3   -0.1    0.94  -0.    -0.562]
CD_x error at height 24:
 [ 0.2   -0.4   -0.1   -0.8   -0.127]
CD_y error at height 24:
 [-0.8    1.328 -3.3    2.5    1.   ]
CD_x error at height 32:
 [-1.2 -2.1 -0.1 -0.  -1. ]
CD_y error at height 32:
 [ 0.1    1.3   -0.029 -0.7   -0.2  ]
CD_x error at height 40:
 [-0.242 -0.2   -0.6    0.1   -2.1  ]
CD_y error at height 40:
 [ 1.8 -0.2 -2.   0.2 -1.3]
CD_x error at height 48:
 [ 0.5    0.535 -0.1   -0.    -1.01 ]
CD_y error at height 48:
 [ 1.    1.8  -1.1  -0.6  -0.05]
CD_x error at height 56:
 [ 0.9    0.652  0.9   -0.     0.465]
CD_y error at height 56:
 [-0.3   0.44  0.   -0.5  -0.02]
CD_x error at height 64:
 [-0.5   1.2  -0.    0.07  2.9 ]
CD_y err

100%|██████████| 20/20 [43:10:01<00:00, 7770.08s/it]  

CD_x error at height 72:
 [2.7 2.6 3.6 4.  2.8]
CD_y error at height 72:
 [-0.1    0.324  0.1   -0.954 -0.5  ]
CD_RMSE for label 19: [1.453 1.041]
CD Error: [2.018 1.952]
RMSE(Mean, Std): [0.008 0.649] [0.003 0.204]
NMSE(Mean, Std): [11.768  1.628] [0.975 0.514]
NMSE_height: [[22.967 17.961 15.726 17.647 17.234 15.744 14.335 13.242 12.314 11.476 10.712 10.021  9.403
   8.853  8.368  7.944  7.574  7.253  6.977  6.739 ...  8.858  9.563 10.412 11.422 12.609 13.99
  15.576 17.382 19.42  21.708 24.256 27.117 30.418 33.727 35.032 33.216 33.269 43.312 59.734
  73.982]
 [ 3.401  2.692  2.314  2.682  2.67   2.483  2.303  2.161  2.036  1.918  1.807  1.703  1.606
   1.518  1.441  1.373  1.313  1.26   1.214  1.174 ...  0.955  0.928  0.912  0.905  0.925  0.97
   1.028  1.102  1.187  1.271  1.358  1.444  1.526  1.528  1.19   0.882  2.148  3.428  4.358
   4.943]]
Top Error (mean): [39.406  2.28 ]


Mean runtime: 1.7660516500473022
RMSE(Ihb, rate): [0.008 0.649]
NRMSE(Ihb, rate): [11.768  1.628]
CD Error(x, y): [2.018 1.952]


In [ ]:
Err_x, Err_y = np.array([]), np.array([])
for tmp in range(len(CD_err_x)):
    Err_x = np.append(Err_x, np.array(CD_err_x[tmp]).flatten())
    Err_y = np.append(Err_y, np.array(CD_err_y[tmp]).flatten())

np.savetxt(opt.respath+"CDErr-x_deepCNN.csv", Err_x.flatten(), delimiter=' ')
np.savetxt(opt.respath+"CDErr-y_deepCNN.csv", Err_y.flatten(), delimiter=' ')
from utils.Res_plot import CD_errXY_plot
CD_errXY_plot(Err_x, Err_y, "DeePCNN's ")